## Experiments: Sensitive Analysis Hyperparameter RSAST:


It is runned RSAST in a set of UCR datasets with a predefined number of runs ("runs"). Then, it is selected a range ("range_total") between 1, 10, 30 ,50 and 100 for the selected dataset.

In [37]:
#configure directory to import sast libraries
import sys 
import os 
#add sast library path
file_path = os.path.expanduser("~/random_sast/sast")
sys.path.append(file_path)
#add cd_diagram library path
file_path = os.path.expanduser("~/random_sast/cd_diagram")
sys.path.append(file_path)

#sys.path

In [38]:
from sast import *
from sktime.datasets import load_UCR_UEA_dataset, tsc_dataset_names
from sktime.classification.kernel_based import RocketClassifier
import time
import pandas as pd
import os

### Reading Datasets

In [39]:
# Set directory where the csv files are located
directory = os.getcwd()+'/ResultsByClassifier'

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        # Append the dataframe to the list
        df['filename']=filename
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_other_methods = pd.concat(dfs, ignore_index=True)
df_other_methods=df_other_methods[['filename','folds:','0']]
df_other_methods['method']=df_other_methods['filename'].str.split('_').str[0]
df_other_methods
df_other_methods=df_other_methods.rename(columns={'folds:':'dataset','0':'score'})
df_other_methods=df_other_methods[['dataset','score','method']]


In [40]:
df_other_methods['method'].unique()

array(['BOSS', 'Catch22', 'cBOSS', 'HIVE-COTEv1', 'InceptionTime',
       'ProximityForest', 'ResNet', 'RISE', 'ROCKET', 'S-BOSS', 'SAST',
       'STC', 'TS-CHIEF', 'TSF', 'WEASEL'], dtype=object)

In [41]:
# Set directory where the csv files are located
directory = os.getcwd()+'/ResultsRsast'
#directory = os.getcwd()+'/results_accuracy_per_ds'

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.startswith("df_overall_tunning") and filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        df['rpoint']=df['rpoint'].astype(str)
        df['nb_per_class']=df['nb_per_class'].astype(str)

        
        # Append the dataframe to the list
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_result = pd.concat(dfs, ignore_index=True)
df_result.head(3)


,Unnamed: 0,accuracy,time,cweights_time,fsubsequence_time,tdataset_time,tclassifier_time,dataset_name,classifier_name,rpoint,nb_per_class,method,len_method
0,0,0.9,22.449903,0.118879,18.913046,3.344095,0.073366,BirdChicken,ACF&PACF: n_random_points=1 nb_inst_per_class=1,1,1,Rsast,ACF&PACF
1,1,0.9,0.423205,0.104959,0.011462,0.302871,0.003022,BirdChicken,ACF: n_random_points=1 nb_inst_per_class=1,1,1,Rsast,ACF
2,2,0.7,18.675146,0.104006,18.243167,0.323289,0.003877,BirdChicken,PACF: n_random_points=1 nb_inst_per_class=1,1,1,Rsast,PACF


In [42]:
#df_result=pd.read_csv("df_overall_tunning_SmoothSubspace3.csv")

df_result['dataset_name'].unique()


array(['BirdChicken', 'Chinatown', 'Coffee',
       'DistalPhalanxOutlineAgeGroup', 'DistalPhalanxOutlineCorrect',
       'GunPoint', 'ItalyPowerDemand', 'ShapeletSim', 'SmoothSubspace',
       'SonyAIBORobotSurface1', 'SonyAIBORobotSurface2',
       'SyntheticControl'], dtype=object)

In [43]:
df_result.columns.unique()

Index(['Unnamed: 0', 'accuracy', 'time', 'cweights_time', 'fsubsequence_time',
       'tdataset_time', 'tclassifier_time', 'dataset_name', 'classifier_name',
       'rpoint', 'nb_per_class', 'method', 'len_method'],
      dtype='object')

### Hyperparameter Tunning RSAST 

#### Accuracy: Subsequence Lenght Method 

In [44]:

filter_param=df_result[df_result.rpoint=='(lenthg ts)//2']
filter_param.rpoint.unique()

array(['(lenthg ts)//2'], dtype=object)

In [45]:

filter_param=filter_param[filter_param.nb_per_class=='(max instances per class)//2']
filter_param.nb_per_class.unique()

array(['(max instances per class)//2'], dtype=object)

In [71]:
# create a pivot table with the mean of score by dataset

len_method_vs_ds=pd.pivot_table(filter_param, values='accuracy', index=['len_method'],columns=['dataset_name'], aggfunc='var')
len_method_vs_ds=np.transpose(len_method_vs_ds)#[['ACF','PACF']]
len_method_vs_ds=len_method_vs_ds.reset_index()
df_rocket=df_other_methods[df_other_methods["method"]=="ROCKET"]
merged_df = len_method_vs_ds.merge(df_rocket,left_on='dataset_name', right_on='dataset',  how='left')
merged_df=merged_df.drop('method',axis=1)
merged_df

,dataset_name,ACF,ACF&PACF,Max ACF,Max PACF,None,PACF,dataset,score
0,Chinatown,0.000210,0.000105,0.000194,0.000075,0.000252,0.000106,Chinatown,0.982507
1,Coffee,0.000000,0.000000,0.001276,0.000255,0.000893,0.000000,Coffee,1.000000
2,DistalPhalanxOutlineCorrect,0.001041,0.000033,0.000148,0.000679,0.000306,0.000016,DistalPhalanxOutlineCorrect,0.771739
3,GunPoint,0.000583,0.000079,0.000595,0.000105,0.000301,0.000145,GunPoint,1.000000
4,ItalyPowerDemand,0.000092,0.000018,0.000117,0.000122,0.000111,0.000049,ItalyPowerDemand,0.968902
5,SmoothSubspace,0.001093,0.000251,0.001726,0.001440,0.001028,0.000849,SmoothSubspace,0.980000
6,SonyAIBORobotSurface1,0.006281,0.000494,0.004843,0.000613,0.001139,0.000369,SonyAIBORobotSurface1,0.921797
7,SonyAIBORobotSurface2,0.003548,0.000091,0.002278,0.000343,0.001508,0.000186,SonyAIBORobotSurface2,0.916055
8,SyntheticControl,0.000175,0.000044,0.000170,0.000069,0.000058,0.000018,SyntheticControl,1.000000


In [47]:
# create a pivot table with the mean of score by number of random points
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['rpoint'], aggfunc='mean')

rpoint,(lenthg ts)//2,1,10,100,30,50
len_method,,,,,,
ACF,0.844550,0.691909,0.759942,0.770735,0.755309,0.768940
ACF&PACF,0.908183,0.771166,0.795179,0.803428,0.796486,0.803360
Max ACF,0.843557,0.682629,0.745901,0.752002,0.729101,0.759011
Max PACF,0.887261,0.702575,0.766164,0.768126,0.750297,0.766269
None,0.879397,0.703778,0.746451,0.747651,0.737234,0.744940
PACF,0.907876,0.774466,0.788911,0.807836,0.805999,0.801066


In [48]:
# create a pivot table with the mean of score by number of instances per class
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
len_method,,,,,,
ACF,0.843292,0.687808,0.746759,0.775434,0.764785,0.773490
ACF&PACF,0.908973,0.761950,0.784456,0.809701,0.805259,0.807348
Max ACF,0.843335,0.648312,0.744717,0.760516,0.750556,0.764797
Max PACF,0.888465,0.685895,0.768388,0.769227,0.758780,0.769763
None,0.885602,0.669947,0.745040,0.759241,0.742791,0.755924
PACF,0.908610,0.765580,0.786661,0.805531,0.812499,0.807166


#### Time: Subsequence Lenght Method 

In [49]:
# create a pivot table with the mean of the time by dataset
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['dataset_name'], aggfunc='mean')

dataset_name,BirdChicken,Chinatown,Coffee,DistalPhalanxOutlineAgeGroup,DistalPhalanxOutlineCorrect,GunPoint,ItalyPowerDemand,ShapeletSim,SmoothSubspace,SonyAIBORobotSurface1,SonyAIBORobotSurface2,SyntheticControl
len_method,,,,,,,,,,,,
ACF,81.630738,0.120657,43.827708,132.824072,231.141162,42.810175,0.674857,7.645123,1.599955,0.333337,0.815245,112.463974
ACF&PACF,297.280662,0.368780,148.320089,419.191777,679.493913,95.588660,1.649303,222.211859,2.648634,1.902398,3.008572,348.782316
Max ACF,2.745482,0.122675,3.134972,49.805655,76.582509,4.674037,0.681721,2.640422,1.595359,0.318064,0.627533,39.817326
Max PACF,107.885490,0.134552,6.470813,29.798286,46.293955,4.200933,0.747006,124.217464,1.849311,0.345630,0.495278,25.971835
None,1.853694,0.109866,1.756151,28.495936,43.753207,2.811864,0.665972,1.836391,1.571106,0.242769,0.397229,24.519854
PACF,208.928276,0.290148,105.678870,293.491555,452.581422,52.941237,1.651981,215.296044,2.606331,1.603944,2.348842,227.328232


In [50]:
# create a pivot table with the mean of the time by number of instances per class
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
len_method,,,,,,
ACF,44.178483,2.373510,23.480767,99.652279,49.324756,69.258616
ACF&PACF,128.400813,9.709713,92.692350,320.521385,172.886178,229.620080
Max ACF,13.266944,0.595672,4.944691,30.789112,13.200776,20.165633
Max PACF,8.485383,2.550219,24.716056,39.871285,33.465675,34.088771
None,7.439471,0.423855,3.110451,18.324744,8.048543,12.049405
PACF,84.789994,7.013095,69.480939,222.058293,120.564248,160.971638


In [51]:
# create a pivot table with the mean of the time by number of random points
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['rpoint'], aggfunc='mean')

rpoint,(lenthg ts)//2,1,10,100,30,50
len_method,,,,,,
ACF,37.947243,1.488241,13.389779,130.968899,39.624738,65.758233
ACF&PACF,108.270144,21.921783,57.223933,410.000808,137.380898,221.968678
Max ACF,9.902502,0.528778,3.969407,38.331110,11.495467,19.226213
Max PACF,6.480317,17.462431,19.094521,42.696064,26.448413,31.288048
None,5.635084,0.339922,2.404092,23.064877,6.909817,11.305819
PACF,70.848307,20.229480,43.447628,280.858725,99.843578,151.683651


#### Variance: Subsequence Lenght Method 

In [52]:
# create a pivot table with the variance of the accuracy by number of instances per class
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['nb_per_class'], aggfunc='var')

nb_per_class,(max instances per class)//2,1,10,100,30,50
len_method,,,,,,
ACF,0.009441,0.040546,0.046647,0.040777,0.041907,0.041345
ACF&PACF,0.006433,0.043382,0.049611,0.043275,0.044455,0.045776
Max ACF,0.010408,0.048201,0.045892,0.046434,0.049268,0.043458
Max PACF,0.009214,0.058034,0.047746,0.051457,0.055003,0.052188
None,0.008964,0.053575,0.052156,0.049738,0.055347,0.049442
PACF,0.006087,0.039678,0.049082,0.045494,0.038881,0.044502


In [53]:
# create a pivot table with the variance of the accuracy by number of random points
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['rpoint'], aggfunc='var')

rpoint,(lenthg ts)//2,1,10,100,30,50
len_method,,,,,,
ACF,0.009864,0.048936,0.042523,0.038073,0.042859,0.039086
ACF&PACF,0.006476,0.050089,0.045282,0.043149,0.045432,0.043676
Max ACF,0.010197,0.050618,0.046303,0.043036,0.056416,0.042723
Max PACF,0.009510,0.062342,0.048683,0.048163,0.058837,0.048640
None,0.010061,0.061299,0.050703,0.047742,0.054937,0.050498
PACF,0.006359,0.043945,0.049580,0.039236,0.041470,0.044007


#### Another Results

In [54]:
df_result[['dataset_name','len_method','classifier_name']].describe()

,dataset_name,len_method,classifier_name
count,7080,7080,7080
unique,12,6,216
top,Chinatown,ACF&PACF,None: n_random_points= (lenthg ts)//2 nb_inst_...
freq,720,1180,110


In [55]:
pd.pivot_table(df_result, values='time', index=['dataset_name'], aggfunc='sum')/(60*60)

,time
dataset_name,
BirdChicken,14.590090
Chinatown,0.038223
Coffee,3.435429
DistalPhalanxOutlineAgeGroup,19.866818
DistalPhalanxOutlineCorrect,38.246154
GunPoint,6.767564
ItalyPowerDemand,0.202361
ShapeletSim,11.955152
SmoothSubspace,0.395690


In [56]:
df_result['time'].sum()/(60*60)

118.62954584048855

#### Generate CD Diagram Tunning Hyperparameter

In [57]:
#create dataset to generate cd diagram
df_perf=pd.pivot_table(df_result, values='accuracy', index=['classifier_name','dataset_name'], aggfunc='mean')
df_perf=df_perf.reset_index()
#df_perf.classifier_name.unique()


In [58]:
param_per_ds=pd.pivot_table(df_perf, index=['dataset_name'],values=['classifier_name'] ,aggfunc='count')
param_per_ds=param_per_ds.reset_index()
complete_ds=param_per_ds[param_per_ds.classifier_name==max(param_per_ds.classifier_name)]["dataset_name"].unique()

df_perf=df_perf[df_perf.dataset_name.isin(complete_ds)]
df_result=df_result[df_result.dataset_name.isin(complete_ds)]
df_perf.dataset_name.unique()

array(['Chinatown', 'Coffee', 'DistalPhalanxOutlineCorrect', 'GunPoint',
       'ItalyPowerDemand', 'SmoothSubspace', 'SonyAIBORobotSurface1',
       'SonyAIBORobotSurface2', 'SyntheticControl'], dtype=object)

In [59]:
from cd_function import *
#generate hyperparameter tuning cd diagrams
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^ACF&PACF:')], labels=True, title="ACF&PACF comparison", name_file='images_cd_diagram/cd-diagram_ACF&PACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^PACF:')], labels=True, title="PACF comparison", name_file='images_cd_diagram/cd-diagram_PACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^ACF:')], labels=True, title="ACF comparison", name_file='images_cd_diagram/cd-diagram_ACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^None:')], labels=True, title="None comparison", name_file='images_cd_diagram/cd-diagram_None.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^Max ACF:')], labels=True, title="Max ACF comparison", name_file='images_cd_diagram/cd-diagram_MaxACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^Max PACF:')], labels=True, title="Max PACF comparison", name_file='images_cd_diagram/cd-diagram_MaxPACF.png')
draw_cd_diagram(df_perf, labels=True, title="overall Hyperparameter Tunning", name_file='images_cd_diagram/cd-diagram_overall_tunning.png')



c:\Users\Surface pro\anaconda3\lib\site-packages\scipy\stats\_morestats.py:3255: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
c:\Users\Surface pro\anaconda3\lib\site-packages\scipy\stats\_morestats.py:3269: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")
c:\Users\Surface pro\anaconda3\lib\site-packages\scipy\stats\_morestats.py:3255: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
c:\Users\Surface pro\anaconda3\lib\site-packages\scipy\stats\_morestats.py:3269: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")
c:\Users\Surface pro\anaconda3\lib\site-packages\scipy\stats

In [60]:
_, average_ranks, _ = wilcoxon_holm(df_perf=df_perf)
min_rank= min(average_ranks)
average_ranks=pd.DataFrame(average_ranks)
average_ranks

c:\Users\Surface pro\anaconda3\lib\site-packages\scipy\stats\_morestats.py:3255: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
c:\Users\Surface pro\anaconda3\lib\site-packages\scipy\stats\_morestats.py:3269: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


,0
ACF: n_random_points=10 nb_inst_per_class=1,188.555556
Max ACF: n_random_points=1 nb_inst_per_class=1,183.333333
ACF: n_random_points=1 nb_inst_per_class=1,183.166667
Max ACF: n_random_points=10 nb_inst_per_class=1,182.722222
Max ACF: n_random_points=30 nb_inst_per_class=1,180.944444
...,...
PACF: n_random_points= (lenthg ts)//2 nb_inst_per_class=50,46.500000
PACF: n_random_points= (lenthg ts)//2 nb_inst_per_class=100,41.222222
ACF&PACF: n_random_points= (lenthg ts)//2 nb_inst_per_class=30,40.388889
ACF&PACF: n_random_points= (lenthg ts)//2 nb_inst_per_class=50,39.777778


In [61]:
#generate example cd diagrams
#draw_cd_diagram(pd.read_csv('example_cd_diagram.csv'), labels=True, title="Example CD Diagram", name_file='cd-diagram_example.png')

#### Focus on most accurate lenght method

##### Overall Accuracy

In [62]:
# highest accurate hyperparameters
average_ranks[average_ranks[0]==min_rank][0].index

Index(['ACF&PACF: n_random_points= (lenthg ts)//2 nb_inst_per_class=100'], dtype='object')

In [63]:
# filter by method with ACF&PACF
# create a pivot table with the mean of score by hyperparameter
df_result_acc=df_result[df_result["len_method"]=="ACF&PACF"]
pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.911620,0.852489,0.905129,0.922922,0.920123,0.923068
1,0.892852,0.686768,0.713458,0.783630,0.786682,0.779326
10,0.911517,0.759709,0.784934,0.792719,0.790797,0.784631
100,0.913162,0.769627,0.779623,0.786626,0.791062,0.789956
30,0.906526,0.739646,0.774026,0.791221,0.788357,0.788395
50,0.907571,0.784185,0.780751,0.799493,0.790667,0.793975


In [64]:
# create a pivot table with the variatioon of score by hyperparameter
pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='var')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.006239,0.008075,0.006450,0.005298,0.005690,0.005705
1,0.008201,0.077835,0.069274,0.052440,0.048668,0.061031
10,0.006059,0.042913,0.052589,0.063066,0.060028,0.065847
100,0.006125,0.043648,0.061938,0.069845,0.065558,0.067514
30,0.007354,0.059284,0.072472,0.063734,0.059688,0.068106
50,0.006625,0.040077,0.058312,0.062046,0.067499,0.067372


##### Overall time complexity

In [65]:
# create a pivot table with the mean of time spent in minutes
pd.pivot_table(df_result_acc, values=['time','cweights_time','fsubsequence_time','tdataset_time','tclassifier_time'], index=['dataset_name'], aggfunc='mean')/60

,cweights_time,fsubsequence_time,tclassifier_time,tdataset_time,time
dataset_name,,,,,
Chinatown,0.000153,0.001468,0.000136,0.004364,0.006146
Coffee,0.001170,0.102598,0.000719,2.367384,2.472001
DistalPhalanxOutlineCorrect,0.000789,0.029408,0.014176,11.280322,11.324899
GunPoint,0.000702,0.035006,0.001057,1.556273,1.593144
ItalyPowerDemand,0.000133,0.002412,0.000269,0.024644,0.027488
SmoothSubspace,0.000103,0.002886,0.000560,0.040557,0.044144
SonyAIBORobotSurface1,0.000361,0.004451,0.000191,0.026673,0.031707
SonyAIBORobotSurface2,0.000332,0.004047,0.000192,0.045540,0.050143
SyntheticControl,0.000680,0.040138,0.260529,5.511458,5.813039


In [75]:
# create a pivot table with the mean of time spent in minutes by random points and instances per class
pivot = pd.pivot_table(df_result_acc, values='time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')/60
pivot = pivot.reindex(columns=["(max instances per class)//2","1","10","30","50","100"], index=["(lenthg ts)//2","1","10","30","50","100"])
pivot

nb_per_class,(max instances per class)//2,1,10,30,50,100
rpoint,,,,,,
(lenthg ts)//2,2.262302,0.066976,0.699961,1.531656,1.971251,2.463671
1,0.079352,0.015704,0.028344,0.067217,0.098734,0.136958
10,0.543001,0.021491,0.202964,0.543210,0.827109,1.166891
30,1.577738,0.061125,0.577847,1.576576,2.423524,3.551959
50,2.648219,0.102516,0.939739,3.180713,4.046529,6.200529
100,5.240316,0.197701,1.920289,5.138023,8.085165,11.684438


##### Calculate weights time complexity

In [67]:
pd.pivot_table(df_result_acc, values='cweights_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.020218,0.021650,0.021994,0.019723,0.018885,0.020628
1,0.023276,0.024096,0.019200,0.028788,0.022187,0.023673
10,0.020068,0.028215,0.021417,0.029950,0.025726,0.025905
100,0.018814,0.033067,0.027229,0.028383,0.032144,0.029564
30,0.020759,0.026214,0.026748,0.030612,0.026857,0.032590
50,0.021405,0.029220,0.028994,0.033902,0.029394,0.032452


##### Finding subsequences time complexity

In [68]:
pd.pivot_table(df_result_acc, values='fsubsequence_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.575341,0.058414,0.503132,0.920188,0.877360,0.871794
1,0.895647,0.209526,0.420297,1.056532,0.812689,0.965449
10,0.625422,0.060313,0.521390,1.329269,0.992600,1.221184
100,0.622080,0.139847,1.324482,4.048829,2.719541,3.563706
30,0.575407,0.078700,0.697913,1.877865,1.430290,1.730007
50,0.552192,0.092177,0.782487,2.507851,1.675952,2.282470


##### Transform Dataset time complexity

In [69]:
pd.pivot_table(df_result_acc, values='tdataset_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,135.010547,3.918334,41.428301,146.737863,90.912501,117.260519
1,3.815847,0.685663,1.179866,7.040353,3.128471,4.854899
10,31.884152,1.155201,11.546528,68.497927,31.460493,48.257812
100,313.498322,11.606075,113.676258,694.817520,304.441514,479.883200
30,93.972544,3.491579,33.847533,209.077693,92.957509,143.285694
50,158.173592,5.923742,55.422165,345.493267,154.461437,240.095836


##### Classifier time complexity

In [70]:
pd.pivot_table(df_result_acc, values='tclassifier_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
rpoint,,,,,,
(lenthg ts)//2,0.124491,0.018569,0.040740,0.131077,0.084045,0.114435
1,0.025339,0.019383,0.080314,0.090405,0.068713,0.078774
10,0.047808,0.044572,0.087531,0.153542,0.112040,0.119493
100,0.272575,0.078819,0.184110,2.155695,1.079162,1.621125
30,0.091255,0.069584,0.097066,2.124120,0.175827,0.357740
50,0.141093,0.102854,0.146934,23.987943,34.670425,0.372042
